# 🧠 CatBoost para Predicción de Salario IT - Comparativa de Estrategias

Este notebook aplica y compara dos enfoques para entrenar un modelo CatBoost:

1. **CatBoost con preprocesamiento manual + GridSearchCV**
2. **CatBoost con variables categóricas nativas (sin preprocesamiento)**

📁 El dataset debe estar en `./data/survey_results_public.csv`


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor, Pool
import warnings
warnings.filterwarnings("ignore")


In [5]:
# Cargar datos
df = pd.read_csv("./data/survey_results_public.csv")

# Renombrar columnas
df = df.rename(columns={
    "Employment": "Tipo_empleo",
    "RemoteWork": "Trabajo_remoto",
    "DevType": "Rol",
    "EdLevel": "Nivel_educativo",
    "YearsCodePro": "Anios_experiencia",
    "Country": "Pais",
    "OrgSize": "Tamano_empresa",
    "ConvertedCompYearly": "Salario_anual"
})

# Filtrar valores extremos
df = df[df["Salario_anual"].between(10000, 300000)]

# Normalizar experiencia
df["Anios_experiencia"] = df["Anios_experiencia"].replace({
    "Less than 1 year": "0", "More than 50 years": "51"
})
df["Anios_experiencia"] = pd.to_numeric(df["Anios_experiencia"], errors="coerce")

# Agrupar países con pocos registros
top_paises = df["Pais"].value_counts().head(15).index
df["Pais"] = df["Pais"].apply(lambda x: x if x in top_paises else "Otro")

# Expandir columna multietiqueta
roles = df["Rol"].str.get_dummies(sep=";")
df = pd.concat([df.drop(columns="Rol"), roles], axis=1)

# Eliminar nulos
df = df.dropna()

# Variables nuevas
df["Salario_anual_log"] = np.log1p(df["Salario_anual"])
df["Anios_experiencia_log"] = np.log1p(df["Anios_experiencia"])
df["Anios_experiencia_cuadrado"] = df["Anios_experiencia"] ** 2

# Ordenar tamaño de empresa
orden = [
    "Just me - I am a freelancer, sole proprietor, etc.",
    "2 to 9 employees",
    "10 to 19 employees",
    "20 to 99 employees",
    "100 to 499 employees",
    "500 to 999 employees",
    "1,000 to 4,999 employees",
    "5,000 to 9,999 employees",
    "10,000 or more employees"
]
df = df[df["Tamano_empresa"].isin(orden)]
df["Tamano_empresa_ordinal"] = df["Tamano_empresa"].apply(lambda x: orden.index(x))

# Guardamos una copia completa para el segundo modelo
df_raw = df.copy()


## 🔧 Estrategia 1: CatBoost + Preprocesamiento + GridSearchCV

In [6]:
y = df["Salario_anual_log"]
X = df.drop(columns=["Salario_anual", "Salario_anual_log", "Tamano_empresa"])


cat_cols = ["Tipo_empleo", "Trabajo_remoto", "Pais", "Nivel_educativo"]
num_cols = [col for col in X.columns if col not in cat_cols]

cat_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

num_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, cat_cols),
    ("num", num_pipeline, num_cols)
])

pipe = Pipeline([
    ("pre", preprocessor),
    ("regressor", CatBoostRegressor(verbose=0, random_state=42))
])

params = {
    "regressor__iterations": [500],
    "regressor__learning_rate": [0.03, 0.05],
    "regressor__depth": [6, 8],
    "regressor__l2_leaf_reg": [1, 3]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

grid = GridSearchCV(pipe, params, cv=3, scoring="r2", verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

print("Mejor configuración:", grid.best_params_)
print("R² (test - preprocesado):", r2_score(y_test, grid.predict(X_test)))


ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

## 🧠 Estrategia 2: CatBoost sin preprocesar (nativo con categóricas)

In [ ]:
X_cb = df_raw.drop(columns=["Salario_anual", "Salario_anual_log"])
y_cb = df_raw["Salario_anual_log"]

cat_features = ["Tipo_empleo", "Trabajo_remoto", "Pais", "Nivel_educativo", "Tamano_empresa"]

X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_cb, y_cb, random_state=42)

pool_train = Pool(X_train_cb, y_train_cb, cat_features=cat_features)
pool_test = Pool(X_test_cb, y_test_cb, cat_features=cat_features)

modelo_cb = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3,
    random_state=42,
    verbose=100
)

modelo_cb.fit(pool_train)
y_pred_cb = modelo_cb.predict(pool_test)

print("R² (test - categóricas nativas):", r2_score(y_test_cb, y_pred_cb))
